<a href="https://colab.research.google.com/github/yvain13/TAPEAgents/blob/main/SN_TAPEAGENTS_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to TapeAgents!


**TapeAgents** is a framework that leverages a structured, replayable log (**Tape**) of the agent session to facilitate all stages of the LLM Agent development lifecycle. In TapeAgents, the agent reasons by processing the tape and the LLM output to produce new thoughts, actions, control flow steps and append them to the tape. The environment then reacts to the agent’s actions by likewise appending observation steps to the tape.

In this tutorial, you will learn:
- how to create TapeAgents using the low-level API
- run and resume TapeAgents
- have one TapeAgent reuse another TapeAgent's tape as training data

In upcoming versions of this tutorial, you will also learn:
- how to make a team TapeAgent with subagents
- how to build TapeAgents using available high-level APIs
- how to build a TapeAgent that streams partial steps

Other tutorials and examples will cover:
- code execution and browser use
- finetuning
- the TapeAgents apps (Studio and Browser)

In [2]:
from google.colab import userdata

# Setup
We're assuming that you already installed the project through the `make setup` or the jupyter notebook is running in the context of the project. If not, please refer to the [README](README.md) for more detailed instructions.

In [3]:
# Now set the OPENAI_API_KEY environment variable to your API key.

import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    # os.environ["OPENAI_ORGANIZATION"] = "" # optional
if "SERPER_API_KEY" not in os.environ:
    os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
today = "2025-01-17"  # fixed date for reproducible tests


# If you prefer to skip the OpenAI setup and not make any LLM calls, you can use ones from the cache.
# it will work instead of the real LLM fine as long as the prompts are not changed.
# Uncomment the following lines to use the cache:
#
# from tapeagents import llms
# import os
# llm_cache_path = "tests/res/intro_notebook/tapedata.sqlite"
# if not os.path.exists(llm_cache_path):
#     llm_cache_path = f"../{llm_cache_path}"
# assert os.path.exists(llm_cache_path)
# llms._REPLAY_SQLITE = llm_cache_path


In [ ]:
!pip install tapeagents

In [ ]:
!pip install markdownify python-pptx Pillow puremagic whisper readability-lxml  lxml_html_clean

# 1. Your first TapeAgent

In this section, we will build the simplest possible "hello world" agent. We will then go through all the new concepts that you need to know to understand the code. This section is quite long, but with the solid foundation you acquire here other TapeAgent tutorials will be easy to process.

Without further ado, here's the code!

In [5]:
from tapeagents.agent import Agent, Node
from tapeagents.core import Prompt, SetNextNode
from tapeagents.dialog_tape import AssistantStep, UserStep, DialogTape
from tapeagents.llms import LLMStream, LiteLLM
from tapeagents.prompting import tape_to_messages

llm = LiteLLM(model_name="gpt-4o-mini")


class MainNode(Node):
    name: str = "main"

    def make_prompt(self, agent: Agent, tape: DialogTape) -> Prompt:
        # Render the whole tape into the prompt, each step is converted to message
        return Prompt(messages=tape_to_messages(tape))

    def generate_steps(self, agent: Agent, tape: DialogTape, llm_stream: LLMStream):
        yield AssistantStep(content=llm_stream.get_text())  # Generate new step from the LLM output stream.
        yield SetNextNode(next_node="main")  # Which node to execute next, more on that later


agent = Agent[DialogTape].create(llm, nodes=[MainNode()])
start_tape = DialogTape(steps=[UserStep(content="Tell me about ServiceNow in 3 sentences")])
final_tape = agent.run(start_tape).get_final_tape()  # agent will start executing the first node
print(f"Final tape: {final_tape.model_dump_json(indent=2)}")


Final tape: {
  "metadata": {
    "id": "58e7a88c-837f-47c9-bfd8-06d3f3e39161",
    "parent_id": "03d2a3be-e9d1-40e4-99c0-4e5a2b9475e1",
    "author": "Agent",
    "author_tape_id": null,
    "n_added_steps": 2,
    "error": null,
    "result": {}
  },
  "context": null,
  "steps": [
    {
      "metadata": {
        "id": "935c3e2c-3104-4f7b-bb04-090d3d0917af",
        "prompt_id": "",
        "node": "",
        "agent": "",
        "other": {}
      },
      "kind": "user",
      "content": "Tell me about ServiceNow in 3 sentences"
    },
    {
      "metadata": {
        "id": "cc5409b3-bd0a-4715-92b8-4c8338a24e7b",
        "prompt_id": "8ba1c49d-fa83-40a8-b8ab-04cb468a9159",
        "node": "main",
        "agent": "Agent",
        "other": {}
      },
      "kind": "assistant",
      "content": "ServiceNow is a cloud-based platform that provides robust IT service management (ITSM) solutions, helping organizations automate and manage their IT services and workflows. It offers a ra

Now let's learn about tapes, steps, prompts, llm streams, nodes and agents.

### Tape

The fundamental concept of the TapeAgents is the `Tape`, a comprehensive semantic level log of the agent's session. A `Tape` contains a context and a sequence of `Step` objects. As you can see, a TapeAgent runs by adding steps (such as `UserStep` or `AssistantStep`) to the _tape_. This example uses the `DialogTape` tape, which is a basic tape for user-assistant conversations. Let's see what are the possible steps in a `DialogTape`.

Some of these steps should be familiar to you. `UserStep`, `AssistantStep`, `SystemStep` and `ToolResult` correspond to `role=user`, `role=assistant`, `role=system` and `role=tool` LLM API messages respectively. `ToolCalls` and `AssistantThought` correspond to assistant messages where the LLM requests a tool call or produces an intermediate thought that is not meant to be shown to the user. `SetNextNode` and `Pass` are TapeAgent's internal step to control which node it should run at the next iteration (more on this below).

### Prompt format; LLMs

We use the industry-standard "chat.completions" prompt format in TapeAgents: a list of user/assistant/system/tool messages plus tool schemas.

The LLMs in TapeAgent take `Prompt` and return an `LLMStream` object. The `LLMStream` object can be used both to fast-forward to the complete response text and to stream partial outputs step by step.

In [6]:
llm_stream = LiteLLM(model_name="gpt-4o-mini-2024-07-18", stream=True)

# Streaming
prompt = Prompt(messages=[{"role": "user", "content": "Write hello world in Java"}])
for event in llm_stream.generate(prompt):
    print(event.chunk, end="")

# No streaming
# (note: you can not use Prompt object for more than 1 LLM call in TapeAgents)
prompt = Prompt(messages=[{"role": "user", "content": "Write hello world in C"}])
print("\n" + "-" * 30)
print(llm_stream.generate(prompt).get_text())


Sure! Here is a simple Java program that prints "Hello, World!" to the console:

```java
public class HelloWorld {
    public static void main(String[] args) {
        System.out.println("Hello, World!");
    }
}
```

To run this program, you would need to:

1. Save the code in a file named `HelloWorld.java`.
2. Open a terminal or command prompt and navigate to the directory where the file is located.
3. Compile the program using the command:
   ```
   javac HelloWorld.java
   ```
4. Run the compiled program using the command:
   ```
   java HelloWorld
   ```

This will output:
```
Hello, World!
```None
------------------------------
Certainly! Here is a simple "Hello, World!" program written in C:

```c
#include <stdio.h>

int main() {
    printf("Hello, World!\n");
    return 0;
}
```

### Explanation:
- `#include <stdio.h>`: This line includes the standard input-output library, which is necessary for using the `printf` function.
- `int main()`: This defines the main function where t

In the example above we use the easiest way to create a prompt from the tapes: `tape_to_messages`. Under the hood, this method uses `step.llm_dict()` method of all non-control steps in the tape to create the prompt:

A key priority in TapeAgents is making use of the data that running the agent generates. To make this possible, some TapeAgent LLMs know how to make their finetuning data:

### Node

A node represents an uninterruptible atom of TapeAgent's computation. When TapeAgents runs a node, it uses its two main functions: `make_prompt` to create an LLM Prompt from the tape and `generate_steps` to create new steps from the LLM output. To build a node, you can subclass `Node` and override these functions. Note that `generate_steps` must be a generator, a design choice we made to make TapeAgents a streaming-friendly framework.

Let's see what the node from the above example can do.

### Agent and its nodes

The TapeAgent agent iteratively runs the nodes and appends the steps generated by each node to the tape. To select which next node to run, internally a TapeAgent computes the **tape view** object. The Tape remains the only **state** that the agent uses, the view only represents its content in a way that is convenient for the agent to use.


In [7]:
from tapeagents.view import TapeViewStack
from tapeagents.core import StepMetadata

# The "top" view in the tape view stack is the view of the current agent.
# Initially `top.last_node` is empty and the agent will run the first node from its list".
tape1 = DialogTape(steps=[UserStep(content="Hi, AI!")])
last_node = TapeViewStack.compute(tape1).top.last_node
print(f"1: {last_node}")
assert last_node == ""


# When the agent computes the view, it updates `top.last_node` with the node from the latest agent step
# The agent will search the next node after the last_node in its nodes list.
tape2 = DialogTape(
    steps=[
        UserStep(content="Hi, AI!"),
        AssistantStep(metadata=StepMetadata(prompt_id="123", node="main"), content="AI here, how I can help?"),
    ]
)
last_node = TapeViewStack.compute(tape2).top.last_node
print(f"2: {last_node}")
assert last_node == "main"

# The SetNextNode step on the tape changes `top.next_node` to the value of the `next_node` field in the SetNextNode step.
# The agent will use this value
tape3 = DialogTape(
    steps=[
        UserStep(content="Hi, AI!"),
        AssistantStep(metadata=StepMetadata(prompt_id="123"), content="AI here, how I can help?"),
        SetNextNode(next_node="act"),
    ]
)
next_node = TapeViewStack.compute(tape3).top.next_node
print(f"3: {next_node}")
assert next_node == "act"


1: 
2: main
3: act


By default the agent stops after the last node has produced an `Action` step. The action steps are the steps by which the agent requests information from the environment. For example, `AssistantStep` is an `Action` as it indicates the agent awaits the user response, `ToolCalls` is an action requesting tool call results. Let's look at all possible steps in `DialogTape` tape and see which of them are actions, observations and thoughts.

In [8]:
from tapeagents.core import Action, Pass, Thought, Observation
from tapeagents.dialog_tape import AssistantThought, ToolCalls, ToolResult

assert all([issubclass(step_class, Action) for step_class in [AssistantStep, ToolCalls]])
assert all([issubclass(step_class, Thought) for step_class in [AssistantThought, SetNextNode, Pass]])
assert all([issubclass(step_class, Observation) for step_class in [UserStep, ToolResult]])


Now we are ready to look at a simplified summary of the corner-stone `agent.run` algorithm.

1. Compute the new tape view
2. Choose the active agent (more on multi-agent TapeAgents later)
3. Choose the active node
4. Run the node and add steps on the tape
5. If the last node yielded an action, then stop, else repeat.

`agent.run` returns an `AgentStream` object which allows iterating through the agent's steps (or partial steps when streaming) and fast-forwardin to the complete new tape with `get_final_tape`.

#### Converse with a TapeAgent

Let's continue the conversation with the agent that previously responded to us with the Vulcan definition from the StarTrek.
Remember, the session is stored in the tape **final_tape**.

In [9]:
tape_to_continue = final_tape + [UserStep(content="I want to know what future hold for this company?")]
continued_tape = agent.run(tape_to_continue).get_final_tape()
print(continued_tape.model_dump_json(indent=2))


{
  "metadata": {
    "id": "fdf3f235-411a-4500-9aa0-89482f3c4758",
    "parent_id": "e289ba05-fa0f-4dfb-a60f-76f256753ebd",
    "author": "Agent",
    "author_tape_id": null,
    "n_added_steps": 2,
    "error": null,
    "result": {}
  },
  "context": null,
  "steps": [
    {
      "metadata": {
        "id": "935c3e2c-3104-4f7b-bb04-090d3d0917af",
        "prompt_id": "",
        "node": "",
        "agent": "",
        "other": {}
      },
      "kind": "user",
      "content": "Tell me about ServiceNow in 3 sentences"
    },
    {
      "metadata": {
        "id": "cc5409b3-bd0a-4715-92b8-4c8338a24e7b",
        "prompt_id": "8ba1c49d-fa83-40a8-b8ab-04cb468a9159",
        "node": "main",
        "agent": "Agent",
        "other": {}
      },
      "kind": "assistant",
      "content": "ServiceNow is a cloud-based platform that provides robust IT service management (ITSM) solutions, helping organizations automate and manage their IT services and workflows. It offers a range of appli

Note that the agent is able to continue talking to you thanks for `SetNextNode(next_node="main")` step that `generate_steps` produced. If you try to remove this step as an exercise, the agent will crash because there is only one node.

#### Tape rendering

LLM agents create a lot of data that can be overwhelming to process. In TapeAgents we render the tape with the associated prompts and outputs into a more readable HTML for you. To make this work, we store prompts and outputs in an SQLite database every time you call `agent.run()`.

Here's how to use tape rendering in the notebook:

In [10]:
from tapeagents.renderers import render_tape_with_prompts
from tapeagents.renderers.pretty import PrettyRenderer
from IPython.display import HTML

HTML(render_tape_with_prompts(continued_tape, PrettyRenderer()))


# 2. Your TapeAgent with planning and tools

Let's build a TapeAgent that plans and acts. We will be using OpenAI function calling capabilities in this example.

In [48]:
from tapeagents.core import SetNextNode
from tapeagents.dialog_tape import AssistantThought, ToolCalls
from tapeagents.environment import ToolEnvironment
from tapeagents.orchestrator import main_loop
from tapeagents.tools.stock import get_stock_data, get_stock_ticker

system_instruction = f"""
You will help the user to learn about financials of companies.
Use as many relevant tools as possible to include more details and facts in your responses.
Today is {today}.
"""
system_message = {"role": "system", "content": system_instruction}

env = ToolEnvironment([get_stock_ticker, get_stock_data])


class PlanNode(Node):
    name: str = "plan"

    def make_prompt(self, agent, tape: DialogTape) -> Prompt:
        guidance = "Write a natural language plan on how to use tools help the user. Output a list of numbered items, like 1., 2., 3., etc."
        guidance_message = {"role": "user", "content": guidance}
        return Prompt(
            messages=[system_message] + tape_to_messages(tape) + [guidance_message], tools=env.get_tool_schema_dicts()
        )

    def generate_steps(self, agent, tape, llm_stream: LLMStream):
        if content := llm_stream.get_output().content:
            yield AssistantThought(content=content)
        else:
            raise ValueError()


class ActNode(Node):
    name: str = "act"

    def make_prompt(self, agent, tape: DialogTape) -> Prompt:
        guidance = "Follow the plan you created to earlier. When you are done, respond to the user."
        guidance_message = {"role": "user", "content": guidance}
        return Prompt(
            messages=[system_message] + tape_to_messages(tape) + [guidance_message], tools=env.get_tool_schema_dicts()
        )

    def generate_steps(self, agent, tape, llm_stream: LLMStream):
        o = llm_stream.get_output()
        if o.content:
            yield AssistantStep(content=o.content)
            yield SetNextNode(next_node="plan")
        elif o.tool_calls:
            yield ToolCalls.from_llm_output(o)
            yield SetNextNode(next_node="act")
        else:
            raise ValueError()


agent1 = Agent.create(LiteLLM(model_name="gpt-4o", parameters={"temperature": 0.1}), nodes=[PlanNode(), ActNode()])

print("Run the agent!")
final_tape1 = None
for event in main_loop(agent1, DialogTape() + [UserStep(content="Is it wise to invest in ServiceNow today?")], env):
    if ae := event.agent_event:
        if ae.step:
            print(ae.step.model_dump_json(indent=2))
        if ae.final_tape:
            final_tape1 = ae.final_tape
    if event.observation:
        print(event.observation.model_dump_json(indent=2))
assert final_tape1
print("Final tape:")
HTML(render_tape_with_prompts(final_tape1, PrettyRenderer()))


Run the agent!
{
  "metadata": {
    "id": "f6e332c1-9126-4658-a6fb-a87104dbe853",
    "prompt_id": "6893b60c-3701-4180-8642-c6be44de02f8",
    "node": "plan",
    "agent": "Agent",
    "other": {}
  },
  "kind": "assistant_thought",
  "content": "1. Identify the stock ticker symbol for ServiceNow by using the `get_stock_ticker` function with the company name \"ServiceNow.\"\n\n2. Once the stock ticker is obtained, use the `get_stock_data` function to retrieve the recent stock price data for ServiceNow. Set the date range to cover the last 6 months to provide a comprehensive view of the stock's recent performance.\n\n3. Analyze the stock price data to identify trends, such as upward or downward movements, volatility, and any significant changes in price.\n\n4. Provide insights based on the stock price analysis, including any patterns or notable events that may have influenced the stock's performance.\n\n5. Offer a summary of the findings to help the user make an informed decision about

INFO:tapeagents.tools.tool_cache:Cache hit for _get_stock_ticker with args ('ServiceNow',) and kwargs {}: True


{
  "metadata": {
    "id": "bc4cb93f-106c-440c-b46d-8e57161e16be",
    "prompt_id": "ffc9a78e-6d33-481c-aadd-ec504eef12b1",
    "node": "act",
    "agent": "Agent",
    "other": {}
  },
  "kind": "assistant",
  "tool_calls": [
    {
      "function": {
        "name": "get_stock_ticker",
        "arguments": "{\"company_name\":\"ServiceNow\"}"
      },
      "id": "call_TFYl6SsKSsV3MQd67aJj8QPX",
      "type": "function"
    }
  ]
}
{
  "metadata": {
    "id": "2de17b29-af79-4907-932b-7d0d3b6778fe",
    "prompt_id": "ffc9a78e-6d33-481c-aadd-ec504eef12b1",
    "node": "act",
    "agent": "Agent",
    "other": {}
  },
  "kind": "set_next_node",
  "next_node": "act"
}
{
  "metadata": {
    "id": "79878771-e6fd-4a9a-8d5c-9f702b523211",
    "prompt_id": "",
    "node": "",
    "agent": "",
    "other": {}
  },
  "kind": "tool",
  "content": "NOW",
  "tool_call_id": "call_TFYl6SsKSsV3MQd67aJj8QPX"
}


INFO:tapeagents.tools.tool_cache:Cache hit for _get_stock_data with args ('NOW', '2024-07-17', '2025-01-17') and kwargs {}: True


{
  "metadata": {
    "id": "55c96ece-ba4f-4f75-bba5-0abfd35d4692",
    "prompt_id": "ee2f4bcc-9b45-4449-b40e-f91d15ecf118",
    "node": "act",
    "agent": "Agent",
    "other": {}
  },
  "kind": "assistant",
  "tool_calls": [
    {
      "function": {
        "name": "get_stock_data",
        "arguments": "{\"symbol\":\"NOW\",\"start_date\":\"2024-07-17\",\"end_date\":\"2025-01-17\"}"
      },
      "id": "call_xdRnG6ZGelOyfEm0egW4U0DP",
      "type": "function"
    }
  ]
}
{
  "metadata": {
    "id": "b8c295fc-35c1-4c0e-89b8-0468e1e7e12a",
    "prompt_id": "ee2f4bcc-9b45-4449-b40e-f91d15ecf118",
    "node": "act",
    "agent": "Agent",
    "other": {}
  },
  "kind": "set_next_node",
  "next_node": "act"
}
{
  "metadata": {
    "id": "827671e5-0132-411f-96db-38552de50839",
    "prompt_id": "",
    "node": "",
    "agent": "",
    "other": {}
  },
  "kind": "tool",
  "content": [
    [
      "2024-07-17",
      736.0700073242188
    ],
    [
      "2024-07-19",
      751.2000122070312

The main new thing in this example is the environment. In TapeAgents framework the environment responds to the agent `Action` steps with `Observation` steps. We expect you to use the environment to encapsulate tool use, retrieval, code execution: everything that is non-deterministic, non-stationary, or computationally heavy. On the contrary, we encourage you to implements the agent's deterministic decision-making in `make_prompt` and `generate_steps` methods.

Here we use a pre-defined `main_loop` orchestrator to run the agent and the environment. `main_loop` is a generator of events that you can use as you wish. You are free to implement your own orchestration paradigm with a fine-grained control over what actions get to be executed.

# 5. Multi-agent teams, deeper dive into view stack

Let's add a colleague to our agent that will help it search the internet!

First thing, we need to give this colleague some tools. In TapeAgents, the agents do not use the environment directly, their interaction with the environment is mediated by in orchestrator such as you application. But the agents should know what tools they can call. And if you are using `main_loop` as the orchestrator, it requires one environment that contains tools of all the agents.

Let's go ahead and define environments:
- the environments of the root agent
- the one of its internet search specialist colleague
- the environment that contains all the tools.

Note that we won't use the first two environments to produce observations, we'll use them only to generate tool schemas for agent.

In [12]:
from tapeagents.tools.simple_browser import SimpleTextBrowser

browser = SimpleTextBrowser()
search_agent_env = ToolEnvironment([browser.get_search_results, browser.get_page, browser.get_next_page])


# We will use the tool choice mechanism to let the main agent call its search specialist agent.
# To this end, we create a mock tool that represents calling the search agent.
def call_search_agent(query: str):
    """Use this tool to ask a fellow AI agent to search for information on the web."""
    pass


main_agent_env = ToolEnvironment([get_stock_ticker, get_stock_data, call_search_agent])
whole_env = ToolEnvironment(
    [get_stock_ticker, get_stock_data, browser.get_search_results, browser.get_page, browser.get_next_page]
)


Before we implement the subagent, let's review the way we do multi-agent communication in TapeAgents:
- when the root agent wants to call its subagent "xyz", it puts `Call(agent_name="xyz")` step on the tape
- at the next iteration, the root agent will compute `TapeViewStack` and delegate to the currently active agent. Right after `Call(agent_name="B")` on top of the stack there will be a new view associated with Agent "xyz". The root agent will delegate to "xyz" to make the prompt and to generate the steps from "xyz"'s current node.
- when "xyz" is done it will put `Respond()` step on the tape. At the next iteration the view stack won't have "xyz"'s view on the top any more.

A reader familiar with the concept of a call stack will find the `TapeViewStack` concept very similar...

Note that for the purpose of computing the view stack the root agent's name is not known. It's view will be signed as "root".

Let's explore the way `Call` and `Respond` works with a minimal example.


In [13]:
from tapeagents.core import Call, Respond

tape = DialogTape(
    steps=[
        UserStep(content="Compute 2 + 2"),
        Call(agent_name="xyz", content="what is 2 + 2, dear xyz"),
        AssistantThought(content="deep thinking by agent xyz"),
        Respond(content="I heard it is 4"),
    ]
)
# We will print a brief summary of view stack after each step
for i in range(0, len(tape)):
    print(f"View stack after step {i}")
    view_stack = TapeViewStack.compute(tape[: i + 1])
    for view in view_stack.stack:
        step_summary = ", ".join([step.__class__.__name__ for step in view.steps])
        print(f"-- {view.agent_full_name}: {step_summary}")
# Note how "root/xyz" view appears after step 1 and disappears after step 3.
# Also note how "root" does not see private thoughts of "xyz" (step 2), and "xyz"
# does not see the initial observation of root (step 0)


View stack after step 0
-- root: UserStep
View stack after step 1
-- root: UserStep, Call
-- root/xyz: Call
View stack after step 2
-- root: UserStep, Call
-- root/xyz: Call, AssistantThought
View stack after step 3
-- root: UserStep, Call, Respond


We are now ready to proceed to the implementation of the internet search agent!

In [14]:
from tapeagents.nodes import FixedStepsNode
from tapeagents.core import Respond
from tapeagents.prompting import view_to_messages

search_system_instruction = "Use at most 5 tool calls to search the request info on on the web."
search_system_message = {"role": "system", "content": search_system_instruction}


class SearchAgentMainNode(Node):
    name: str = "main"

    def make_prompt(self, agent, tape: DialogTape) -> Prompt:
        view = agent.compute_view(tape)
        return Prompt(messages=view_to_messages(view.top, agent), tools=search_agent_env.get_tool_schema_dicts())

    def generate_steps(self, agent, tape, llm_stream: LLMStream):
        o = llm_stream.get_output()
        if o.content:
            # if the LLM responds, yield Respond(..) as your last step
            yield Respond(content=o.content)
        elif o.tool_calls:
            # when the LLM suggests tool calls, yield them as action steps
            yield ToolCalls.from_llm_output(o)
            yield SetNextNode(next_node="main")
        else:
            raise ValueError()


search_agent = Agent.create(
    name="search_agent",
    llms=LiteLLM(model_name="gpt-4o", parameters={"temperature": 0.1}),
    nodes=[SearchAgentMainNode()],
)
# To test the subagent, we'll make a mock root agent that immediately calls "search_agent"
call_step = Call(agent_name="search_agent", content="What influenced ServiceNow stock price in late 2024?")
test_root_agent = Agent.create(subagents=[search_agent], nodes=[FixedStepsNode(steps=[call_step])])
start_tape = DialogTape()
final_tape = None
for event in main_loop(test_root_agent, start_tape, search_agent_env):
    # We need to stop the loop when "search_agent" responds,
    # otherwise the code will crash because `test_root_agent` only has one node.
    if (ae := event.agent_event) and isinstance(ae.step, Respond):
        final_tape = ae.partial_tape
        break
assert final_tape
HTML(render_tape_with_prompts(final_tape, PrettyRenderer()))


INFO:tapeagents.tools.simple_browser:Search ServiceNow stock price influence late 2024 not in cache
INFO:tapeagents.tools.simple_browser:Page https://www.investors.com/research/ibd-stock-of-the-day/servicenow-stock/?mod=hp_minor_pos32 not in cache
INFO:tapeagents.tools.simple_browser:Page https://www.yahoo.com/news/m/d4907c3b-229d-3d0f-9cf7-1737f8e55b5f/servicenow-stock-is.html not in cache
INFO:tapeagents.tools.simple_browser:Page https://www.marketbeat.com/originals/servicenow-targets-new-highs-with-ai-and-automation/ not in cache
INFO:tapeagents.tools.simple_browser:Page https://www.investing.com/news/analyst-ratings/servicenow-stock-target-lifted-maintains-buy-amid-renewal-cycle-optimism-93CH-3739318 not in cache
INFO:tapeagents.tools.simple_browser:Page https://www.servicenow.com/company/media/press-room/servicenow-third-quarter-2024-financial-results.html not in cache


KeyboardInterrupt: 

Finally, let's add the search subagent to a financial analyst agent like the ones in earlier examples. We need to give the root agent a way to use the LLM to decide whether to call the search specialist, and if yes, what query to pass to it. We will abuse the tool calling mechanism for this purpose to make the example simpler.

In [20]:
import json
from tapeagents.core import SetNextNode
from tapeagents.dialog_tape import AssistantThought, ToolCalls
from tapeagents.orchestrator import MainLoopStatus, main_loop
from tapeagents.view import Call
from IPython.display import clear_output

system_instruction = f"""
You will help the user to learn about financials of companies.
For general user queries, include some info about stock price changes during the last year, as well as some general information on the company.
Today is {today}.
"""
system_message = {"role": "system", "content": system_instruction}


class PlanNode(Node):
    name: str = "plan"

    def make_prompt(self, agent, tape) -> Prompt:
        view = agent.compute_view(tape)
        guidance = "Write a natural language plan on how to use tools help the user. Output a list of numbered items, like 1., 2., 3., etc."
        guidance_message = {"role": "user", "content": guidance}
        return Prompt(
            messages=[system_message] + view_to_messages(view.top, agent) + [guidance_message],
            tools=main_agent_env.get_tool_schema_dicts(),
        )

    def generate_steps(self, agent, dialog, llm_stream: LLMStream):
        if content := llm_stream.get_output().content:
            yield AssistantThought(content=content)
        else:
            raise ValueError()


class ActNode(Node):
    name: str = "act"

    def make_prompt(self, agent, tape: DialogTape) -> Prompt:
        view = agent.compute_view(tape)
        guidance = "Follow the plan you created to earlier. When you are done, respond to the user."
        guidance_message = {"role": "user", "content": guidance}
        return Prompt(
            messages=[system_message] + view_to_messages(view.top, agent) + [guidance_message],
            tools=main_agent_env.get_tool_schema_dicts(),
        )

    def generate_steps(self, agent, dialog, llm_stream: LLMStream):
        o = llm_stream.get_output()
        if o.content:
            yield SetNextNode(next_node="plan")
            yield AssistantStep(content=o.content)
        elif o.tool_calls:
            yield SetNextNode(next_node="act")
            # only keep the tool calls before the call to another agent
            agent_call = None
            for i, tc in enumerate(o.tool_calls):
                if tc.function.name == "call_search_agent":
                    agent_call = tc
                    o.tool_calls = o.tool_calls[:i]
                    break
            # either produce the ToolCalls action OR call another agent
            if o.tool_calls:
                yield ToolCalls.from_llm_output(o)
            else:
                assert agent_call and agent_call.function.name == "call_search_agent"
                yield Call(agent_name="search_agent", content=json.loads(agent_call.function.arguments)["query"])

        else:
            raise ValueError()


multi_agent_analyst = Agent.create(
    name="analyst",
    subagents=[search_agent.clone()],
    llms=LiteLLM(model_name="gpt-4o", parameters={"temperature": 0.1}),
    nodes=[PlanNode(), ActNode()],
)

print("Run the agent!")
start_tape = DialogTape(steps=[UserStep(content=" How is the performance of Nvidia ?")])
for event in main_loop(multi_agent_analyst, start_tape, whole_env):
    # This agent runs for a while, so we will show you a fresh render every time
    # when the environment finishes reacting with new actions
    if new_tape := event.agent_tape or event.env_tape:
        clear_output()
        display(HTML(render_tape_with_prompts(new_tape, PrettyRenderer())))
    # Uncomment this if you want to pause after every loop
    #if event.env_tape:
         #input("Press Enter the run the next iteration of the main loop")
    if event.status == MainLoopStatus.EXTERNAL_INPUT_NEEDED:
        break


Look at this rather long tape and note how "analyst" calls "search_agent", and how the latter then responds to "analyst". Congratulations, you now know how to build a multi-agent TapeAgent!

In [ ]:
!pip install ffmpeg-python webvtt-py whisper yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 35.0 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 10.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803373 sha256=7047680a5aff80ac009c2ee9e8151d5b5473448dfd25c4fbaf64f66fe9ecafc0
  Stored in directory: /root/.cache/pip/wheels/2f/f2/ce/6eb23db4091d026238ce76703bd66da60b969d70bcc81d5d3a
Successfully built openai-whisper


In [ ]:
from tapeagents.tools.media_reader import get_video_observation

In [ ]:
url = "https://www.youtube.com/watch?v=kN93lrS1nfw"
output_dir = "/content/"
start_time = "00:00:00"  # Optional: specify the start time
end_time = "00:01:00"  # Optional: specify the end time

video_observation = get_video_observation(url, output_dir, start_time, end_time)
print(video_observation)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

metadata=StepMetadata(id='7d87b0f2-5a43-4208-8939-c4cdcf2e1369', prompt_id='', node='', agent='', other={}) kind='video_observation' local_dir='/content/' video_path='/content/kN93lrS1nfw_00-00-00_00-01-00.mp4' video_contact_sheet_paths=['/content/kN93lrS1nfw_00-00-00_00-01-00_contact-sheet_1.png'] thumbnail_path='/content/kN93lrS1nfw.webp' subtitle_path='/content/kN93lrS1nfw_00-00-00_00-01-00.vtt' subtitle_text="00:00:00.000: Google is making a lot of serious moves in AI, and recently launching Gemini Cheap and Serial\n00:00:03.920: just shocked the world. You know, at first, I was skeptical, but honestly speaking, after trying it,\n00:00:09.520: this is impressive. So in this video, I will share what makes Gemini 2 unique, its limitations,\n00:00:15.380: and some of the best ways to use it over ChatGPT or other AI tools like perplexity. Let's go.\n00:00:22.480: What makes Gemini 2 truly unique compared to other AIs, it has multi-model capability.\n00:00:27.560: Unlike ChatGPT, which 

In [22]:
!pip install pysnc

In [23]:
from pysnc import ServiceNowClient

In [24]:
client = ServiceNowClient('dev260789', ('admin', userdata.get('snP')))

In [54]:

gr = client.GlideRecord('incident')
gr.limit = 50
gr.fields = [ 'short_description','Description','priority','impact','urgency','category',]
gr.query()
for r in gr:
	print(r)

incident({'sys_id': '0c5f3cece1b12010f877971dea0b1449', 'short_description': 'ATF:TEST2', 'urgency': '3', 'impact': '3', 'priority': '5', 'category': 'inquiry'})
incident({'sys_id': '1c741bd70b2322007518478d83673af3', 'short_description': 'Unable to connect to email', 'urgency': '2', 'impact': '2', 'priority': '3', 'category': 'inquiry'})
incident({'sys_id': '1c832706732023002728660c4cf6a7b9', 'short_description': 'My computer is not detecting the headphone device', 'urgency': '2', 'impact': '2', 'priority': '3', 'category': 'Hardware'})
incident({'sys_id': '46b66a40a9fe198101f243dfbc79033d', 'short_description': 'Reset my password', 'urgency': '1', 'impact': '1', 'priority': '1', 'category': 'inquiry'})
incident({'sys_id': '46b9490da9fe1981003c938dab89bda3', 'short_description': 'Need Oracle 10GR2 installed', 'urgency': '3', 'impact': '2', 'priority': '4', 'category': 'database'})
incident({'sys_id': '46c03489a9fe19810148cd5b8cbf501e', 'short_description': 'Need new Blackberry set up'

In [30]:

import pandas as pd
df = pd.DataFrame(gr.to_pandas())

In [57]:
info = df.iloc[19].to_string()
id = df.iloc[19].sys_id

In [58]:
print(info)

sys_id                                46f3ee0ea9fe198100c5c0e53d5abe0b
short_description    My disk is still having issues. Can't delete a...
priority__value                                                      5
priority__display                                         5 - Planning
impact__value                                                        3
impact__display                                                3 - Low
urgency__value                                                       3
urgency__display                                               3 - Low
category__value                                                inquiry
category__display                                       Inquiry / Help


In [59]:
import json
from tapeagents.core import SetNextNode
from tapeagents.dialog_tape import AssistantThought, ToolCalls
from tapeagents.orchestrator import MainLoopStatus, main_loop
from tapeagents.view import Call
from IPython.display import clear_output

system_instruction = f"""
You will resolve the user queries based on the short descrition and description of the incident . USe Search agent to gather the information and provide correct
result to the user.
Today is {today}.
"""
system_message = {"role": "system", "content": system_instruction}


class PlanNode(Node):
    name: str = "plan"

    def make_prompt(self, agent, tape) -> Prompt:
        view = agent.compute_view(tape)
        guidance = "Write a natural language plan on how to use tools help the user. Output a list of numbered items, like 1., 2., 3., etc."
        guidance_message = {"role": "user", "content": guidance}
        return Prompt(
            messages=[system_message] + view_to_messages(view.top, agent) + [guidance_message],
            tools=main_agent_env.get_tool_schema_dicts(),
        )

    def generate_steps(self, agent, dialog, llm_stream: LLMStream):
        if content := llm_stream.get_output().content:
            yield AssistantThought(content=content)
        else:
            raise ValueError()


class ActNode(Node):
    name: str = "act"

    def make_prompt(self, agent, tape: DialogTape) -> Prompt:
        view = agent.compute_view(tape)
        guidance = "Follow the plan you created to earlier. When you are done, respond to the user."
        guidance_message = {"role": "user", "content": guidance}
        return Prompt(
            messages=[system_message] + view_to_messages(view.top, agent) + [guidance_message],
            tools=main_agent_env.get_tool_schema_dicts(),
        )

    def generate_steps(self, agent, dialog, llm_stream: LLMStream):
        o = llm_stream.get_output()
        if o.content:
            yield SetNextNode(next_node="plan")
            yield AssistantStep(content=o.content)
        elif o.tool_calls:
            yield SetNextNode(next_node="act")
            # only keep the tool calls before the call to another agent
            agent_call = None
            for i, tc in enumerate(o.tool_calls):
                if tc.function.name == "call_search_agent":
                    agent_call = tc
                    o.tool_calls = o.tool_calls[:i]
                    break
            # either produce the ToolCalls action OR call another agent
            if o.tool_calls:
                yield ToolCalls.from_llm_output(o)
            else:
                assert agent_call and agent_call.function.name == "call_search_agent"
                yield Call(agent_name="search_agent", content=json.loads(agent_call.function.arguments)["query"])

        else:
            raise ValueError()


multi_agent_analyst = Agent.create(
    name="analyst",
    subagents=[search_agent.clone()],
    llms=LiteLLM(model_name="gpt-4o", parameters={"temperature": 0.1}),
    nodes=[PlanNode(), ActNode()],
)

print("Run the agent!")
start_tape = DialogTape(steps=[UserStep(content=" Here is the info related to the incident :" + info)])
for event in main_loop(multi_agent_analyst, start_tape, whole_env):
    # This agent runs for a while, so we will show you a fresh render every time
    # when the environment finishes reacting with new actions
    if new_tape := event.agent_tape or event.env_tape:
        clear_output()
        display(HTML(render_tape_with_prompts(new_tape, PrettyRenderer())))
        for step in new_tape.steps:
            if isinstance(step, AssistantStep):
                assistant_response = step.content
    # Uncomment this if you want to pause after every loop
    #if event.env_tape:
         #input("Press Enter the run the next iteration of the main loop")
    if event.status == MainLoopStatus.EXTERNAL_INPUT_NEEDED:
        break


In [60]:
print(assistant_response)

Based on the incident details you provided, it seems you're experiencing issues with your disk and are unable to delete files. Here are some troubleshooting steps that might help resolve the issue:

1. **Check File Permissions**: Ensure you have the necessary permissions to delete the file. You may need to change the file or folder ownership. You can find more information on this [Microsoft Community page](https://answers.microsoft.com/en-us/windows/forum/all/external-hard-drive-issue-cant-delete-files/7ac6d127-f017-4454-a070-b16b1b5a78c9).

2. **Run Chkdsk Utility**: If the file system is corrupted, you might not be able to delete files. Running the Chkdsk utility can help correct any file system errors. Detailed instructions can be found on the [Microsoft documentation](https://learn.microsoft.com/en-us/troubleshoot/windows-server/backup-and-storage/cannot-delete-file-folder-on-ntfs-file-system).

3. **Force Delete Files**: Use command line tools or safe mode to force delete files th

In [61]:
gr = client.GlideRecord('incident')
gr.get(id)
True
gr.work_notes = assistant_response
gr.update()

GlideElement('46f3ee0ea9fe198100c5c0e53d5abe0b')